## Librairies

In [ ]:
import getpass
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib as mpl
from utils import plot_image

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubStatisticalDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubStatistical,
    Geometry,
    parse_time,
)

## Accès

In [ ]:
# OAuth ID : sh-642fc01d-e2be-4009-9217-2858c30a82e5
# OAuth Secret : AIr9YI3AlVRw2Gt8771iF7DqMRYUhX4I

## Configuration

In [ ]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
#config.sh_base_url = "https://dataspace.copernicus.eu/browser/?zoom=10&lat=21.18697&lng=-98.12645&themeId=DEFAULT-THEME&visualizationUrl=https%3A%2F%2Fsh.dataspace.copernicus.eu%2Fogc%2Fwms%2Fa91f72b5-f393-4320-bc0f-990129bd9e63&datasetId=S2_L2A_CDAS&fromTime=2023-09-03T00%3A00%3A00.000Z&toTime=2023-09-03T23%3A59%3A59.999Z&layerId=3_NDVI&demSource3D=%22MAPZEN%22&cloudCoverage=30"
config.save("hackathon_arbres")

## Coordonnées de la zone

In [ ]:
coordonnees5 = (-97.509730,20.867649,-97.326817,20.935804) # Zone Alamo Mexico 
coordonnees6 = (-97.509730,20.867649,-97.486817,20.885804) # zoom x1
coordonnees7 = (-97.509730,20.867649,-97.496817,20.875804) # zoom x2
coordonnees8 = (-82.456154,27.676055,-82.443302,27.665115) # Zone Ruskin Florida
coordonnees9 = (-82.471119,27.684740,-82.440068,27.658626) # Zone Finale

## Paramètres Images

In [ ]:
resolution = 10

zone_bbox = BBox(bbox=coordonnees9, crs=CRS.WGS84)
zone_size = bbox_to_dimensions(zone_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {zone_size} pixels")

## Génération de dates selon un Intervalle et le nombre d'Images

In [ ]:
start = datetime.datetime(2023, 1, 1)
end = datetime.datetime(2023, 12, 31)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)
print(f'On a {len(slots)} images')

## Configuration des Composantes Couleur

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

evalscript_true_color_ndwi = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B03", "B08"]
            }],
            output: {
                bands: 2
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B03, sample.B08];
    }
"""

evalscript_true_color_ndvi = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B04", "B08"]
            }],
            output: {
                bands: 2
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B08];
    }
"""

## Fonction pour demander les images du Serveur Copernicus

In [ ]:
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A.define_from(
                    "s2l2a-bis", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=zone_bbox,
        size=zone_size,
        config=config,
    )

## Telechargement des données

In [ ]:
# create a list of requests
list_of_requests = [get_true_color_request(slot) for slot in slots]
list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)

## Affichage des Images en True Color

In [ ]:
# some stuff for pretty plots
ncols = 2
nrows = 6
aspect_ratio = zone_size[0] / zone_size[1]
subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw, squeeze=False)

for idx, image in enumerate(data):
    ax = axs[idx // ncols][idx % ncols]
    ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
    ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

plt.tight_layout()

## Affichage NDWI

In [ ]:
ncols = 2
nrows = 6
aspect_ratio = zone_size[0] / zone_size[1]
subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw, squeeze=False)

for idx, image in enumerate(data):
    # Calculate NDWI for each image
    def calculate_ndwi(image):
        green = image[:, :, 0].astype(float)  # Green channel
        nir = image[:, :, 1].astype(float)  # Nir channel

        # Calculate a "fake" NDVI for RGB images
        ndwi = (green - nir) / (green + nir)
        return ndwi

    ndwi = calculate_ndwi(image)

    # Display the NDWI map in a subplot
    ax = axs[idx // ncols][idx % ncols]
    im = ax.imshow(ndwi, cmap='RdYlGn', vmin=-1, vmax=1)  # Adjust the colormap as needed
    ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

#Add colorbar for the entire figure
cax = fig.add_axes([0.95, 0.15, 0.02, 0.7])  # Adjust position as needed
cax.get_xaxis().set_visible(False)
cax.get_yaxis().set_ticks([])
cax.get_yaxis().labelpad = 15
cax.set_ylabel('NDWI', rotation=90)

plt.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.show()

## Affichage NDVI

In [ ]:
ncols = 2
nrows = 6
aspect_ratio = zone_size[0] / zone_size[1]
subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw, squeeze=False)

for idx, image in enumerate(data):
    # Calculate NDVI for each image
    def calculate_ndvi(image):
        red = image[:, :, 0].astype(float)  # Red channel
        nir = image[:, :, 1].astype(float)  # Nir channel

        # Calculate a "fake" NDVI for RGB images
        ndvi = (nir - red) / (nir + red)
        return ndvi

    ndvi = calculate_ndvi(image)

    # Display the NDvI map in a subplot
    ax = axs[idx // ncols][idx % ncols]
    im = ax.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)  # Adjust the colormap as needed
    ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

#Add colorbar for the entire figure
cax = fig.add_axes([0.95, 0.15, 0.02, 0.7])  # Adjust position as needed
cax.get_xaxis().set_visible(False)
cax.get_yaxis().set_ticks([])
cax.get_yaxis().labelpad = 15
cax.set_ylabel('NDVI', rotation=90)

plt.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.show()

## Insertion data dans Dataframe

In [ ]:
# Convertissez chaque image en une séquence unidimensionnelle pour chaque ligne du DataFrame
data_df = [image.flatten() for image in data]

# Créez un DataFrame à partir des données
df = pd.DataFrame(data_df)

# Affichez le DataFrame
print(df)

## Application KMeans

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df)
    wcss.append(kmeans.inertia_)

# Tracez le coude
plt.plot(range(1, 11), wcss)
plt.title('Méthode du Coude')
plt.xlabel('Nombre de clusters')
plt.ylabel('WCSS')  # Within-Cluster Sum of Squares
plt.show()

# Segmentation

In [ ]:
# CLUSTER 2
# Prétraitement de l'image
image = np.clip(data[4] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means
k_means = KMeans(n_clusters=2, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée")

plt.show()

In [ ]:
# CLUSTER 4
# Prétraitement de l'image
image = np.clip(data[4] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 4 clusters
k_means = KMeans(n_clusters=4, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 4 Clusters")

plt.show()

In [ ]:
# CLUSTER 10
# Prétraitement de l'image
image = np.clip(data[4] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 10 clusters
k_means = KMeans(n_clusters=10, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 10 Clusters")

plt.show()

In [ ]:
# Image 1
# Prétraitement de l'image
image = np.clip(data[1] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 5 clusters
k_means = KMeans(n_clusters=5, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 5 Clusters")

plt.show()

In [ ]:
# Image 3
# Prétraitement de l'image
image = np.clip(data[3] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 5 clusters
k_means = KMeans(n_clusters=5, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 5 Clusters")

plt.show()

In [ ]:
# Image 4
# Prétraitement de l'image
image = np.clip(data[4] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 5 clusters
k_means = KMeans(n_clusters=5, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 5 Clusters")

plt.show()

In [ ]:
# Image 5
# Prétraitement de l'image
image = np.clip(data[5] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 5 clusters
k_means = KMeans(n_clusters=5, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 5 Clusters")

plt.show()

In [ ]:
# Image 6
# Prétraitement de l'image
image = np.clip(data[6] * 2.5 / 255, 0, 1)
X = image.reshape((-1, 3))

# Utilisation de K-means avec 5 clusters
k_means = KMeans(n_clusters=5, random_state=42)
k_means.fit(X)

# Obtenir les labels et remodeler
X_cluster = k_means.labels_
X_cluster = X_cluster.reshape(image.shape[0], image.shape[1])

# Affichage de l'image originale et de l'image segmentée
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image Originale")

plt.subplot(1, 2, 2)
plt.imshow(X_cluster, cmap="viridis")
plt.title("Image Segmentée avec 5 Clusters")

plt.show()